In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import spacy
import requests
from bs4 import BeautifulSoup

In [4]:
# function for downloading content from the web
def contentDownloader(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.content, "html5lib")
    for item in soup.find_all("div", {"class": "postArticle-content js-postField js-notesSource js-trackedPost"}):
        return item.text

In [6]:
# cleaning the content and return sentences it's not compelete and need to be modified
def cleaning_text(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    list_of_sentences = []
    for sentence in doc.sents:
        list_of_sentences.append(sentence)
    return list_of_sentences

In [7]:
# function for loading diffrenet module
def loading_module(module_url):
    # Import the Universal Sentence Encoder's TF Hub module
    embed = hub.Module(module_url)
    return embed

In [9]:
# function for runinng embedding module on text
def run_embedding(embed_object, text):
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed_object(text))

    return message_embeddings

In [10]:
# function for calculating similarity between question and text
def calculating_text_similarity(question, text, module_url):
    question = cleaning_text(question)
    text = cleaning_text(text)
    question_tensor = tf.Variable(tf.convert_to_tensor(run_embedding(loading_module(module_url), question)))
    text_tensor = tf.Variable(tf.convert_to_tensor(run_embedding(loading_module(module_url), text)))
    multiply_tensor = tf.matmul(question_tensor, text_tensor, transpose_b = True)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        answer = sess.run(multiply_tensor)
    return answer